In [1]:
from collections import deque
from enum import Enum
from heapq import heapify, heappop, heappush, nsmallest
from itertools import batched
from random import shuffle
from typing import Callable, Iterator, Sequence

# Grid size (3x3 for 8-puzzle)
N = 3

### Actions

In [2]:
class Action(Enum):
    LEFT = 1
    RIGHT = 2
    UP = 3
    DOWN = 4

    def __str__(self) -> str:
        return self.name

    def reverse(self) -> 'Action':
        """Return the opposite action."""
        match self:
            case Action.LEFT:
                return Action.RIGHT
            case Action.RIGHT:
                return Action.LEFT
            case Action.UP:
                return Action.DOWN
            case Action.DOWN:
                return Action.UP

### States

In [3]:
State = tuple[int, ...]


def is_valid_list(list_values: Sequence[int]) -> bool:
    """Check if a list of values is valid for an N * N puzzle."""
    # Check if the length is correct
    if len(list_values) != N * N:
        return False

    # Verify that all and only the required values are present
    return set(list_values) == set(range(N * N))


def is_solvable(list_values: Sequence[int]) -> bool:
    """Check if a puzzle configuration is solvable by counting the number of inversions.

    Only valid for grids with an odd N.
    For even-width grids, the solvability formula must also account for the row number
    of the blank tile."""
    num_inversions = 0

    for i, val in enumerate(list_values):
        if val != 0:
            for val_2 in list_values[i + 1 :]:
                if (val_2 != 0) and val_2 < val:
                    num_inversions += 1

    return (num_inversions % 2) == 0


def from_list(list_values: Sequence[int]) -> State:
    """Create a puzzle state from a list of values."""
    if not is_valid_list(list_values):
        raise ValueError(
            f'{list_values} is not a valid list for a grid of size {N} * {N}'
        )

    if not is_solvable(list_values):
        raise ValueError(f'{list_values} is not solvable')

    return tuple(list_values)


def random_state() -> State:
    """Generate a random solvable puzzle state."""
    while True:
        list_values = [i for i in range(N * N)]
        shuffle(list_values)
        if is_solvable(list_values):
            return tuple(list_values)


def is_goal_state(state: State) -> bool:
    """Check if a state is a solution."""
    for i, val in enumerate(state):
        if i != val:
            return False

    return True


def display(state: State) -> None:
    """Display the grid in a visual format."""
    for row in batched(state, N):
        print(list(row))

In [4]:
def coord_blank_cell(state: State) -> tuple[int, int]:
    """Get the (row, col) coordinates of the blank tile."""
    blank_index = state.index(0)
    return blank_index // N, blank_index % N


def get_actions(state: State) -> list[Action]:
    """Get all valid actions from the current state."""
    row, col = coord_blank_cell(state)
    actions = []

    if 0 < col:
        # Blank cell not in the first column
        actions.append(Action.LEFT)
    if col < N - 1:
        # Blank cell not in the last column
        actions.append(Action.RIGHT)
    if 0 < row:
        # Blank cell not in the first row
        actions.append(Action.UP)
    if row < N - 1:
        # Blank cell not in the last row
        actions.append(Action.DOWN)

    return actions


def get_actions_generator(state: State) -> Iterator[Action]:
    """Return lazily all valid actions from the current state using a generator."""
    row, col = coord_blank_cell(state)

    if 0 < col:
        # Blank cell not in the first column
        yield Action.LEFT
    if col < N - 1:
        # Blank cell not in the last column
        yield Action.RIGHT
    if 0 < row:
        # Blank cell not in the first row
        yield Action.UP
    if row < N - 1:
        # Blank cell not in the last row
        yield Action.DOWN


def apply_action(state: State, action: Action) -> tuple[int, ...]:
    """Apply an action to a state and return the resulting state."""
    blank_index = state.index(0)

    # Based on the action, find the index of the cell that will be swapped
    match action:
        case Action.LEFT:
            target_index = blank_index - 1
        case Action.RIGHT:
            target_index = blank_index + 1
        case Action.UP:
            target_index = blank_index - N
        case Action.DOWN:
            target_index = blank_index + N

    # Swap blank cell with target index
    result = list(state)
    result[blank_index], result[target_index] = (
        result[target_index],
        result[blank_index],
    )
    return tuple(result)

### Node Class

In [5]:
class Node:
    """Represents a node in the search tree."""

    id: int = 0

    def __init__(
        self,
        state: tuple[int, ...],
        parent: 'Node | None' = None,
        action: Action | None = None,
        path_cost: int = 0,
        id: int | None = None,
    ) -> None:
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

        # Allow to reset the count to a specific value by specifying a value,
        # otherwise auto-increment
        if id is not None:
            Node.id = id
        self.id: int = Node.id
        Node.id += 1

    def expand(self) -> list['Node']:
        """Generate all valid child nodes from this node, excluding reverse moves."""
        children = []
        action_parent = self.action

        for action in get_actions(self.state):
            # Skip actions that reverse the parent action, like doing LEFT then RIGHT
            if action_parent is not None and action == action_parent.reverse():
                continue

            new_state = apply_action(self.state, action)
            children.append(
                Node(
                    state=new_state,
                    parent=self,
                    action=action,
                    path_cost=self.path_cost + 1,
                )
            )

        return children

    def expand_generator(self) -> Iterator['Node']:
        """Generate lazily all valid child nodes from this node using a generator."""
        action_parent = self.action

        for action in get_actions_generator(self.state):
            # Skip actions that reverse the parent action, like doing LEFT then RIGHT
            if action_parent is not None and action == action_parent.reverse():
                continue

            new_state = apply_action(self.state, action)
            yield Node(
                state=new_state,
                parent=self,
                action=action,
                path_cost=self.path_cost + 1,
            )

    def is_cycle(self) -> bool:
        """Check if current state creates a cycle in the path. Used in the DFS."""
        current_node = self.parent

        while current_node is not None:
            if current_node.state == self.state:
                return True
            current_node = current_node.parent

        return False

    def get_path(self) -> list[Action]:
        """Reconstruct the path of actions from root to this node."""
        path = []
        current_node = self

        while current_node.parent is not None:
            if current_node.action is not None:
                path.append(current_node.action)
            current_node = current_node.parent

        return list(reversed(path))

    def format_path(self) -> str:
        """Get string representation of the action path."""
        return ' -> '.join(str(action) for action in self.get_path())

    def __repr__(self) -> str:
        parent_id = -1 if self.parent is None else self.parent.id
        return f'{self.id=},\n{self.action=!s},\n{parent_id=},\n{self.path_cost=},\nself.state={list(self.state)}'

    def __eq__(self, other: object) -> bool:
        """Overrides the default implementation"""
        if not isinstance(other, Node):
            return NotImplemented
        return self.state == other.state

    def __hash__(self) -> int:
        """Overrides the default implementation"""
        return hash(self.state)

    def __lt__(self, other: object):
        """Overrides the default implementation so that in the PriorityQueue, in case of
        equality for the value of h, node with an smaller id are picked first"""
        if not isinstance(other, Node):
            raise TypeError(
                f'unsupported operand for <: {type(self).__name__} and {type(other).__name__}'
            )
        return self.id < other.id

### Uninformed Search Algorithms

In [6]:
def bfs_with_set(initial_state: State) -> Node | None:
    """Breadth-First Search using a set to keep track of reached states."""
    root_node = Node(state=initial_state, id=0)
    frontier = deque([root_node])
    reached = {initial_state}

    if is_goal_state(root_node.state):
        return root_node

    while len(frontier) > 0:
        node = frontier.popleft()

        for child in node.expand():
            if is_goal_state(child.state):
                return child

            elif child.state not in reached:
                frontier.append(child)
                reached.add(child.state)

    return None


def dfs_with_set(initial_state: State) -> Node | None:
    """Depth-First Search using a set to keep track of reached states."""
    root_node = Node(state=initial_state, id=0)
    frontier = deque([root_node])
    reached = {initial_state}

    while len(frontier) > 0:
        node = frontier.pop()

        if is_goal_state(node.state):
            return node

        for child in node.expand_generator():
            if child.state not in reached:
                frontier.append(child)
                reached.add(child.state)

    return None


def depth_limited_search(initial_state: State, depth_limit: int) -> Node | None:
    """Depth-Limited Search with cycle checking."""
    root_node = Node(state=initial_state, id=0)
    frontier = deque([root_node])

    while len(frontier) > 0:
        node = frontier.pop()

        if is_goal_state(node.state):
            return node

        if node.path_cost < depth_limit:
            for child in node.expand_generator():
                if not child.is_cycle():
                    frontier.append(child)

    return None


def depth_limited_search_with_set(
    initial_state: State, limit_depth: int
) -> Node | None:
    """Depth-Limited Search using a set to keep track of reached states."""
    root_node = Node(state=initial_state, id=0)
    frontier = deque([root_node])

    # Use a dictionary to store the state and the shallowest depth we encountered it
    reached_at_depths: dict[State, int] = {initial_state: 0}

    while len(frontier) > 0:
        node = frontier.pop()

        if is_goal_state(node.state):
            return node

        if node.path_cost < limit_depth:
            for child in node.expand_generator():
                # Only explore a state if we haven't already seen it,
                # or if we're reaching it at a shallower depth
                if (
                    child.state not in reached_at_depths
                    or child.path_cost < reached_at_depths[child.state]
                ):
                    frontier.append(child)
                    reached_at_depths[child.state] = child.path_cost

    return None


def iterative_deepening_search(
    initial_state: State, max_depth: int = 30
) -> Node | None:
    """Iterative Deepening Depth-First Search."""
    for depth in range(1, max_depth):
        result = depth_limited_search(initial_state, depth)
        if result is not None:
            return result

    return None


def iterative_deepening_search_with_set(
    initial_state: State, max_depth: int = 30
) -> Node | None:
    """Iterative Deepening Depth-First Search using a set to keep track of reached states."""
    for depth in range(1, max_depth):
        result = depth_limited_search_with_set(initial_state, depth)
        if result is not None:
            return result

    return None


def bidirectional_bfs_with_set(
    initial_state: State, goal_state: State
) -> tuple[Node, Node] | None:
    """Bidirectional Breadth-First Search.

    Searches simultaneously from initial and goal states, meeting in the middle."""
    node_forward = Node(state=initial_state, id=0)
    node_backward = Node(state=goal_state)

    frontier_forward = deque([node_forward])
    frontier_backward = deque([node_backward])

    reached_forward: dict[State, Node] = {initial_state: node_forward}
    reached_backward: dict[State, Node] = {goal_state: node_backward}

    while frontier_forward and frontier_backward:
        # Forward search
        node = frontier_forward.popleft()
        for child in node.expand():
            if child.state in reached_backward:
                return child, reached_backward[child.state]

            if child.state not in reached_forward:
                frontier_forward.append(child)
                reached_forward[child.state] = child

        # Backward search
        node = frontier_backward.popleft()
        for child in node.expand():
            if child.state in reached_forward:
                return reached_forward[child.state], child

            if child.state not in reached_backward:
                frontier_backward.append(child)
                reached_backward[child.state] = child

    return None


def join_nodes(forward_node: Node, backward_node: Node) -> list[Action]:
    """Reconstitute the complete path from initial state to goal state
    given the two nodes where forward and backward searches met.
    """
    # Get the forward path (from initial state to meeting point)
    forward_path = forward_node.get_path()

    # Get the backward path (from meeting point to goal state)
    # and reverse the actions
    backward_path = [action.reverse() for action in reversed(backward_node.get_path())]

    # Combine both paths
    return forward_path + backward_path

### Heuristic Functions

In [7]:
def num_misplaced_tiles(state: State) -> int:
    """Heuristic that counts the number of misplaced tiles"""
    counter = 0

    for i, val in enumerate(state):
        if i != val:
            counter += 1

    return counter


def manhattan_distance(state: State) -> int:
    """Manhattan heuristic"""
    distance = 0

    for i, val in enumerate(state):
        distance += abs((val // N) - (i // N)) + abs((val % N) - (i % N))

    return distance

### Informed Search Algorithms

In [8]:
def greedy_best_first_search_with_set(
    initial_state: State, heuristic: Callable[[State], int]
) -> Node | None:
    """Greedy Best-First Search using a set to keep track of reached states."""
    root_node = Node(state=initial_state, id=0)
    frontier: list[tuple[int, Node]] = [(heuristic(initial_state), root_node)]
    reached = {initial_state}

    while len(frontier) > 0:
        _, node = heappop(frontier)

        if is_goal_state(node.state):
            return node

        for child in node.expand_generator():
            if child.state not in reached:
                heappush(frontier, (heuristic(child.state), child))
                reached.add(child.state)

    return None


def a_star(initial_state: State, heuristic: Callable[[State], int]) -> Node | None:
    """A* search."""

    def f(node: Node) -> int:
        return node.path_cost + heuristic(node.state)

    root_node = Node(state=initial_state, id=0)
    frontier: list[tuple[int, Node]] = [(f(root_node), root_node)]

    while len(frontier) > 0:
        _, node = heappop(frontier)

        if is_goal_state(node.state):
            return node

        for child in node.expand_generator():
            heappush(frontier, (f(child), child))

    return None


def a_star_with_set(
    initial_state: State, heuristic: Callable[[State], int]
) -> Node | None:
    """A* search using a set to keep track of reached states.

    Optimal only for Consistent heuristics. It assumes that the first time you visit a
    state, you have found the shortest path to it."""
    root_node = Node(state=initial_state, id=0)
    frontier: list[tuple[int, Node]] = [(0 + heuristic(initial_state), root_node)]
    reached: set[State] = set()

    while len(frontier) > 0:
        _, node = heappop(frontier)

        # Skip if we've already expanded this state
        if node.state in reached:
            continue

        if is_goal_state(node.state):
            return node

        # Mark as expanded
        reached.add(node.state)

        # Generate children
        for child in node.expand_generator():
            if child.state not in reached:
                f_value = child.path_cost + heuristic(child.state)
                heappush(frontier, (f_value, child))

    return None


def a_star_with_set_v2(
    initial_state: State, heuristic: Callable[[State], int]
) -> Node | None:
    """A* search using a set to keep track of reached states.

    General implementation for Admissible (but Inconsistent) heuristics.
    It accounts for the possibility that the algorithm might find a "shorter path" to a
    node it has already visited, and updates the cost, and puts the node back in the
    frontier."""
    root_node = Node(state=initial_state, id=0)
    frontier: list[tuple[int, Node]] = [(0 + heuristic(initial_state), root_node)]

    # Use a dictionary to store the state and the shallowest depth we encountered it
    reached_at_depths: dict[State, int] = {}

    while len(frontier) > 0:
        _, node = heappop(frontier)

        # Skip if we've already expanded this state
        if node.state in reached_at_depths:
            continue

        if is_goal_state(node.state):
            return node

        # Mark as expanded
        reached_at_depths[node.state] = node.path_cost

        # Generate children
        for child in node.expand_generator():
            # Only explore a state if we haven't already seen it,
            # or if we're reaching it at a shallower depth
            if (
                child.state not in reached_at_depths
                or child.path_cost < reached_at_depths[child.state]
            ):
                f_value = child.path_cost + heuristic(child.state)
                heappush(frontier, (f_value, child))

    return None


def weighted_a_star_with_set(
    initial_state: State, heuristic: Callable[[State], int], weight: int
) -> Node | None:
    """A* search using a set to keep track of reached states."""
    root_node = Node(state=initial_state, id=0)
    frontier: list[tuple[int, Node]] = [(0 + heuristic(initial_state), root_node)]
    reached: set[State] = set()

    while len(frontier) > 0:
        _, node = heappop(frontier)

        # Skip if we've already expanded this state
        if node.state in reached:
            continue

        if is_goal_state(node.state):
            return node

        # Mark as expanded
        reached.add(node.state)

        # Generate children
        for child in node.expand_generator():
            if child.state not in reached:
                f_value = child.path_cost + weight * heuristic(child.state)
                heappush(frontier, (f_value, child))

    return None


def beam_search_with_set(
    initial_state: State, heuristic: Callable[[State], int], k_size: int
) -> Node | None:
    """Beam search using a set to keep track of reached states.

    We keep only the k nodes with the best f-scores, discarding any other expanded nodes.
    """

    def f(node: Node) -> int:
        return node.path_cost + heuristic(node.state)

    root_node = Node(state=initial_state, id=0)
    frontier: list[tuple[int, Node]] = [(f(root_node), root_node)]
    reached = {initial_state}

    while len(frontier) > 0:
        _, node = heappop(frontier)

        if is_goal_state(node.state):
            return node

        for child in node.expand_generator():
            if child.state not in reached:
                heappush(frontier, (f(child), child))
                if len(frontier) > k_size:
                    # Select the actual k best nodes
                    frontier = nsmallest(k_size, frontier)
                    heapify(frontier)

    return None

### Bidirectional A* Search.

In [9]:
def num_misplaced_tiles_between(state1: State, state2: State) -> int:
    """Count misplaced tiles between two arbitrary states."""
    counter = 0

    for val1, val2 in zip(state1, state2):
        if val1 != val2:
            counter += 1

    return counter


def manhattan_distance_between(state1: State, state2: State) -> int:
    """Calculate Manhattan distance between two arbitrary states."""
    distance = 0

    for i, val in enumerate(state1):
        # Find where this value should be in state2
        target_pos = state2.index(val)

        # Calculate Manhattan distance for this tile
        current_row, current_col = i // N, i % N
        target_row, target_col = target_pos // N, target_pos % N
        distance += abs(current_row - target_row) + abs(current_col - target_col)

    return distance

In [10]:
def bidirectional_a_star_with_set(
    initial_state: State, goal_state: State, heuristic: Callable[[State, State], int]
) -> tuple[Node, Node] | None:
    """Bidirectional A* Search.

    Not correct so far, because it stops the moment the two frontiers meet.
    We should not return immediately when frontiers meet and continues until no better
    solution is possible.
    We should also pick the next node to expand from the minimum value between both
    queues."""
    node_forward = Node(state=initial_state, id=0)
    node_backward = Node(state=goal_state)

    frontier_forward: list[tuple[int, Node]] = [
        (0 + heuristic(initial_state, goal_state), node_forward)
    ]
    frontier_backward: list[tuple[int, Node]] = [
        (0 + heuristic(goal_state, initial_state), node_backward)
    ]

    reached_forward: dict[State, Node] = {}
    reached_backward: dict[State, Node] = {}

    while len(frontier_forward) > 0 and len(frontier_backward) > 0:
        # Forward search
        _, node = heappop(frontier_forward)

        # Skip if we've already expanded this state
        if node.state in reached_forward:
            continue

        # Mark as expanded
        reached_forward[node.state] = node

        if node.state in reached_backward:
            return node, reached_backward[node.state]

        # Generate children
        for child in node.expand():
            if child.state not in reached_forward:
                f_value = child.path_cost + heuristic(child.state, goal_state)
                heappush(frontier_forward, (f_value, child))

        # Backward search
        _, node = heappop(frontier_backward)

        # Skip if we've already expanded this state
        if node.state in reached_backward:
            continue

        # Mark as expanded
        reached_backward[node.state] = node

        if node.state in reached_forward:
            return reached_forward[node.state], node

        # Generate children
        for child in node.expand():
            if child.state not in reached_backward:
                f_value = child.path_cost + heuristic(child.state, initial_state)
                heappush(frontier_backward, (f_value, child))

    return None

In [11]:
# r = random_state()
r = from_list([0, 4, 6, 8, 1, 7, 3, 2, 5])
display(r)

[0, 4, 6]
[8, 1, 7]
[3, 2, 5]


#### Breadth-First Search

In [12]:
res = bfs_with_set(r)
res

self.id=254061,
self.action=LEFT,
parent_id=221351,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [13]:
assert res is not None
res.format_path()

'DOWN -> RIGHT -> RIGHT -> UP -> LEFT -> LEFT -> DOWN -> RIGHT -> DOWN -> RIGHT -> UP -> LEFT -> UP -> RIGHT -> DOWN -> LEFT -> DOWN -> RIGHT -> UP -> LEFT -> LEFT -> DOWN -> RIGHT -> UP -> UP -> LEFT'

#### Depth-First Search

In [14]:
res = dfs_with_set(r)
res

self.id=74903,
self.action=LEFT,
parent_id=74902,
self.path_cost=38954,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

### Depth-Limited Search

In [15]:
# res = depth_limited_search(r, 30)
# res

In [16]:
res = depth_limited_search_with_set(r, 30)
res

self.id=674666,
self.action=LEFT,
parent_id=674664,
self.path_cost=30,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

#### Iterative Deepening Depth-First Search

In [17]:
# res = iterative_deepening_search(r)
# res

In [18]:
res = iterative_deepening_search_with_set(r)
res

self.id=426190,
self.action=LEFT,
parent_id=426189,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

#### Bidirectional Breadth-First Search

In [19]:
goal_state = from_list([i for i in range(N * N)])
res = bidirectional_bfs_with_set(r, goal_state)
assert res is not None
forward_node, backward_node = res
forward_node, backward_node

(self.id=5121,
 self.action=UP,
 parent_id=3192,
 self.path_cost=13,
 self.state=[1, 0, 4, 7, 8, 2, 3, 5, 6],
 self.id=4859,
 self.action=LEFT,
 parent_id=3010,
 self.path_cost=13,
 self.state=[1, 0, 4, 7, 8, 2, 3, 5, 6])

In [20]:
path = join_nodes(forward_node, backward_node)
print(' -> '.join(str(a) for a in path))

DOWN -> RIGHT -> RIGHT -> UP -> LEFT -> LEFT -> DOWN -> RIGHT -> DOWN -> RIGHT -> UP -> LEFT -> UP -> RIGHT -> DOWN -> LEFT -> DOWN -> RIGHT -> UP -> LEFT -> LEFT -> DOWN -> RIGHT -> UP -> UP -> LEFT


#### Greedy Best-First Search

In [21]:
res = greedy_best_first_search_with_set(r, num_misplaced_tiles)
res

self.id=3659,
self.action=LEFT,
parent_id=3658,
self.path_cost=74,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [22]:
res = greedy_best_first_search_with_set(r, manhattan_distance)
res

self.id=849,
self.action=LEFT,
parent_id=848,
self.path_cost=66,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

#### A* Search

In [23]:
res = a_star(r, num_misplaced_tiles)
res

self.id=218761,
self.action=LEFT,
parent_id=218501,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [24]:
res = a_star(r, manhattan_distance)
res

self.id=16709,
self.action=LEFT,
parent_id=16708,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [25]:
res = a_star_with_set(r, num_misplaced_tiles)
res

self.id=65079,
self.action=LEFT,
parent_id=65069,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [26]:
res = a_star_with_set_v2(r, num_misplaced_tiles)
res

self.id=65079,
self.action=LEFT,
parent_id=65069,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [27]:
res = a_star_with_set(r, manhattan_distance)
res

self.id=8941,
self.action=LEFT,
parent_id=8940,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [28]:
res = a_star_with_set_v2(r, manhattan_distance)
res

self.id=8941,
self.action=LEFT,
parent_id=8940,
self.path_cost=26,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [29]:
res = weighted_a_star_with_set(r, manhattan_distance, weight=3)
res

self.id=249,
self.action=UP,
parent_id=247,
self.path_cost=32,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

#### Beam search

In [30]:
res = beam_search_with_set(r, manhattan_distance, k_size=20)
res

self.id=873,
self.action=UP,
parent_id=871,
self.path_cost=34,
self.state=[0, 1, 2, 3, 4, 5, 6, 7, 8]

#### Bidirectional A* Search.

Not working so far, path cost should be 26, not 28.

In [31]:
res = bidirectional_a_star_with_set(r, goal_state, manhattan_distance_between)
res

(self.id=1698,
 self.action=DOWN,
 parent_id=1691,
 self.path_cost=18,
 self.state=[1, 6, 2, 3, 4, 5, 0, 7, 8],
 self.id=2583,
 self.action=LEFT,
 parent_id=1556,
 self.path_cost=10,
 self.state=[1, 6, 2, 3, 4, 5, 0, 7, 8])

In [32]:
path = join_nodes(forward_node, backward_node)
print(' -> '.join(str(a) for a in path))

DOWN -> RIGHT -> RIGHT -> UP -> LEFT -> LEFT -> DOWN -> RIGHT -> DOWN -> RIGHT -> UP -> LEFT -> UP -> RIGHT -> DOWN -> LEFT -> DOWN -> RIGHT -> UP -> LEFT -> LEFT -> DOWN -> RIGHT -> UP -> UP -> LEFT
